In [1]:
import os
from tensorflow.keras.layers import BatchNormalization,Conv2D,Conv2DTranspose,LeakyReLU
from tensorflow.keras.layers import Activation,Flatten,Dense,Reshape,Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as k
import numpy as np

In [2]:
class ConvAutoencoder:
    @staticmethod
    def build(width,height,depth,filters=(32,64),latentDim = 16):
        inputShape = (height,width,depth)
        chandim = -1
        inputs = Input(shape=inputShape)
        x=inputs
        for f in filters:
            x=Conv2D(f,(3,3),strides=2,padding = "same")(x)
            x=LeakyReLU(alpha=0.2)(x)
            x=BatchNormalization(axis = chandim)(x)
        volumeSize = k.int_shape(x)
        x=Flatten()(x)
        latent = Dense(latentDim)(x)
        encoder = Model(inputs,latent,name="encoder")
        
        latentInputs = Input(shape=(latentDim,))
        x = Dense(np.prod(volumeSize[1:]))(latentInputs)
        x= Reshape((volumeSize[1],volumeSize[2],volumeSize[3]))(x)
        for f in filters[::-1]:
            x = Conv2DTranspose(f,(3,3),strides=2,padding="same")(x)
            x= LeakyReLU(alpha=0.2)(x)
            x=BatchNormalization(axis=chandim)(x)
            
        x = Conv2DTranspose(depth,(3,3),padding="same")(x)
        outputs = Activation("sigmoid")(x)
        
        decoder = Model(latentInputs,outputs,name="decoder")
        
        autoencoder = Model(inputs,decoder(encoder(inputs)),name = "autoencoders")
        
        return (encoder,decoder,autoencoder)
        

In [3]:
import matplotlib
matplotlib.use("Agg")

In [4]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2

In [13]:
def build_unsupervised_dataset(data, labels, validLabel=1,anomalyLabel=3,
                               contam=0.01, seed=42):
    validIdxs = np.where(labels == validLabel)[0]
    anomalyIdxs = np.where(labels == anomalyLabel)[0]
    random.shuffle(validIdxs)
    random.shuffle(anomalyIdxs)
    
    i = int(len(validIdxs)*contam)
    anomalyIdxs = anomalyIdxs[:1]
    
    validImages = data[validIdxs]
    anomalyImages = data[anomalyIdxs]
    
    images = np.vstack([validImages,anomalyImages])
    np.random.seed(seed)
    np.random.shuffle(images)
    
    return images
    

In [14]:
def visual_predictions(decoded,gt,samples=10):
    outputs = None
    for i in range(0,samples):
        original = (gt[i] * 255).astype("uint8")
        recon = (decoded[i] * 255).astype("uint8")
        output = np.hstack([original,recon])
        if outputs is None:
            outputs = output
        else:
            outputs = np.vstack([outputs,output])
    return outputs

In [15]:
dataset = os.getcwd()

In [34]:
Vis = os.getcwd()
Vis = Vis+"/recon_vis.png"
print(Vis)

C:\Users\Sujay J\Desktop\PYImage\my exersize\Anomally Detection/recon_vis.png


In [23]:
EPOCHS = 20
INIT_LR = 1e-3
BS = 32

((trainX,trainY),(testX,testY))= mnist.load_data()

In [24]:
images = build_unsupervised_dataset(trainX,trainY,validLabel=1,anomalyLabel=3,contam=0.01)

In [25]:
images = np.expand_dims(images,axis = -1)

In [26]:
images = images.astype("float32")/255.0

In [27]:
(trainX,testX) = train_test_split(images,test_size = 0.2,random_state = 42)

In [28]:
(encoder,decoder,autoencoder) = ConvAutoencoder.build(28,28,1)
opt = Adam(lr = INIT_LR,decay = INIT_LR/EPOCHS)
autoencoder.compile(loss = "mse",optimizers = opt)

In [29]:
h = autoencoder.fit(trainX,trainX,validation_data=(testX,testX),epochs=EPOCHS,batch_size=BS)

Train on 5394 samples, validate on 1349 samples
Epoch 1/20
5394/5394 [==============================] - 27s 5ms/sample - loss: 0.0308 - val_loss: 0.0587
Epoch 2/20
5394/5394 [==============================] - 21s 4ms/sample - loss: 0.0052 - val_loss: 0.0393
Epoch 3/20
5394/5394 [==============================] - 21s 4ms/sample - loss: 0.0040 - val_loss: 0.0104
Epoch 4/20
5394/5394 [==============================] - 20s 4ms/sample - loss: 0.0033 - val_loss: 0.0031
Epoch 5/20
5394/5394 [==============================] - 20s 4ms/sample - loss: 0.0030 - val_loss: 0.0032
Epoch 6/20
5394/5394 [==============================] - 20s 4ms/sample - loss: 0.0027 - val_loss: 0.0030
Epoch 7/20
5394/5394 [==============================] - 21s 4ms/sample - loss: 0.0026 - val_loss: 0.0028
Epoch 8/20
5394/5394 [==============================] - 20s 4ms/sample - loss: 0.0024 - val_loss: 0.0023
Epoch 9/20
5394/5394 [==============================] - 21s 4ms/sample - loss: 0.0023 - val_loss: 0.0024
Epoch 1

In [35]:
decoded = autoencoder.predict(testX)
vis = visual_predictions(decoded,testX)
cv2.imwrite(Vis,vis)

True

In [38]:
N = np.arange(0,EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N,h.history["loss"],label="train_loss")
plt.plot(N,h.history["val_loss"],label = "val_loss")
plt.title("training loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.legend(loc="lower left")
plt.savefig("plot.png")


In [44]:
f = open("dataset", "wb")
f.write(pickle.dumps(images))
f.close()

In [45]:
autoencoder.save("model", save_format="h5")